In [145]:
import os, sys, json, datetime, re  # Provides OS-dependent functionality, system-specific parameters, JSON handling, and date/time manipulation
import pandas as pd             # Provides data structures and data analysis tools
import numpy as np              # Supports large, multi-dimensional arrays and matrices
import requests
import time
from tqdm import tqdm
import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [146]:
data = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Desktop\student data.xlsx"
names_file = r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Desktop\sara_data_regions_and_districts.xlsx"

In [147]:
names = pd.read_excel(names_file)
names.columns

names['Districts'] = names['Districts'].str.strip().str.lower()
names = names.iloc[:,[1,0]]
print(names.to_string())
    

           Districts              Region
0            buffalo   Western New York 
1      niagara falls   Western New York 
2          jamestown   Western New York 
3        canandaigua   The Finger Lakes 
4           onondaga   The Finger Lakes 
5          rochester   The Finger Lakes 
6         binghamton  The Southern Tier 
7             elmira  The Southern Tier 
8            corning  The Southern Tier 
9             ithaca  The Southern Tier 
10           hornell  The Southern Tier 
11      watkins glen  The Southern Tier 
12          syracuse   Central New York 
13          cortland   Central New York 
14       lake placid      North Country 
15       lake george      North Country 
16       plattsburgh      North Country 
17             utica      Mohawk Valley 
18       cooperstown      Mohawk Valley 
19            albany   Capital District 
20  saratoga springs   Capital District 
21      poughkeepsie      Hudson Valley 
22          kingston      Hudson Valley 
23             b

In [148]:
df = pd.read_excel(data)

df.head()


,SCHOOL YEAR,DATE OF REPORT,COUNTY,STATE DISTRICT ID,DISTRICT NAME,SUBGROUP CODE,SUBGROUP NAME,total sum_calc,PK12 TOTAL,PK (HALF DAY),...,GRADE 5,GRADE 6,UNGRADED (ELEMENTARY),GRADE 7,GRADE 8,GRADE 9,GRADE 10,GRADE 11,GRADE 12,UNGRADED (SECONDARY)
0,2021-22,2022-06-30,ALBANY,1.010001e+10,ALBANY,4.0,American Indian/Alaska Native,38,19.0,0.0,...,2.0,2.0,1.0,0.0,1.0,3.0,1.0,3.0,1.0,0.0
1,2021-22,2022-06-30,ALBANY,1.010001e+10,ALBANY,5.0,Black,7830,3915.0,0.0,...,256.0,248.0,24.0,293.0,284.0,324.0,292.0,301.0,314.0,63.0
2,2021-22,2022-06-30,ALBANY,1.010001e+10,ALBANY,6.0,Hispanic,3498,1749.0,0.0,...,135.0,117.0,4.0,144.0,131.0,150.0,133.0,97.0,102.0,17.0
3,2021-22,2022-06-30,ALBANY,1.010001e+10,ALBANY,7.0,Asian/Pacific Islander,1622,811.0,0.0,...,64.0,61.0,2.0,48.0,59.0,60.0,52.0,51.0,52.0,8.0
4,2021-22,2022-06-30,ALBANY,1.010001e+10,ALBANY,8.0,White,3230,1615.0,0.0,...,131.0,89.0,3.0,106.0,123.0,118.0,116.0,107.0,114.0,9.0


In [149]:

groups_keys = df.loc[:,['SUBGROUP NAME', 'SUBGROUP CODE']]

print(f'before: {len(groups_keys)}')
print(f'after: {len(groups_keys.drop_duplicates())}')
print(groups_keys.drop_duplicates())
# list(set(df['SUBGROUP NAME'].to_list()))
# print(*group_names, sep = "\n")

before: 3949
after: 7
                      SUBGROUP NAME  SUBGROUP CODE
0     American Indian/Alaska Native            4.0
1                             Black            5.0
2                          Hispanic            6.0
3            Asian/Pacific Islander            7.0
4                             White            8.0
5                       Multiracial            9.0
3948                              x            NaN


In [150]:
# df.head()
df = df.iloc[:,:8]
df['DISTRICT NAME'] = df['DISTRICT NAME'].str.strip().str.lower()

df


,SCHOOL YEAR,DATE OF REPORT,COUNTY,STATE DISTRICT ID,DISTRICT NAME,SUBGROUP CODE,SUBGROUP NAME,total sum_calc
0,2021-22,2022-06-30,ALBANY,1.010001e+10,albany,4.0,American Indian/Alaska Native,38
1,2021-22,2022-06-30,ALBANY,1.010001e+10,albany,5.0,Black,7830
2,2021-22,2022-06-30,ALBANY,1.010001e+10,albany,6.0,Hispanic,3498
3,2021-22,2022-06-30,ALBANY,1.010001e+10,albany,7.0,Asian/Pacific Islander,1622
4,2021-22,2022-06-30,ALBANY,1.010001e+10,albany,8.0,White,3230
...,...,...,...,...,...,...,...,...
3944,2021-22,2022-06-30,YATES,6.808010e+11,dundee,6.0,Hispanic,46
3945,2021-22,2022-06-30,YATES,6.808010e+11,dundee,7.0,Asian/Pacific Islander,2
3946,2021-22,2022-06-30,YATES,6.808010e+11,dundee,8.0,White,1216
3947,2021-22,2022-06-30,YATES,6.808010e+11,dundee,9.0,Multiracial,44


In [151]:
# print(names['Districts'])

In [152]:

df_w_regions = pd.merge(df, names, left_on= 'DISTRICT NAME', right_on="Districts", how='left')

df_w_regions = df_w_regions.dropna(subset='Districts').reset_index(drop=True)
df_w_regions = df_w_regions.loc[:,['Districts', 'Region', 'SUBGROUP NAME','STATE DISTRICT ID', 'SUBGROUP CODE', 'COUNTY', 'total sum_calc']]
df_w_regions


soc_df = df_w_regions.groupby('Region', as_index=False).agg(
    soc_sum=('total sum_calc', lambda x: (x[df_w_regions.loc[x.index, 'SUBGROUP CODE'] != 8]).sum()),
    total_sum=('total sum_calc', lambda x: x.sum()),
    id_list=('STATE DISTRICT ID', lambda x: x.astype(int).tolist())  # Convert ID column to string and store as list

    # soc_percent=('total sum_calc', lambda x: (x[df_w_regions.loc[x.index, 'SUBGROUP CODE'] != 8]).sum())
)

soc_df['%_students_of_color'] = (soc_df['soc_sum']/soc_df['total_sum'])
soc_df.head()



,Region,soc_sum,total_sum,id_list,%_students_of_color
0,Capital District,15846,29602,"[10100010000, 10100010000, 10100010000, 101000...",0.535302
1,Central New York,31170,42904,"[110200010000, 110200010000, 110200010000, 110...",0.726506
2,Hudson Valley,13872,20754,"[131500010000, 131500010000, 131500010000, 131...",0.668401
3,Long Island,9320,14698,"[280506060000, 280506060000, 280506060000, 280...",0.634100
4,Mohawk Valley,14488,21050,"[412300010000, 412300010000, 412300010000, 412...",0.688266


In [153]:
teach_data = pd.read_excel(r"C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Desktop\staff-race-and-ethnicity-data-2018-2023.xlsx")
teach_data = teach_data[teach_data['FTPT_STATUS'] != "Less Then Full-Time"]


In [154]:

teach_data_by_dis = teach_data.groupby(['STATE_DISTRICT_ID'], as_index=False).aggregate({'White': 'sum','Total': 'sum'})

teach_data_by_dis['toc total'] = teach_data_by_dis['Total'] - teach_data_by_dis['White']
teach_data_by_dis['%_toc'] = teach_data_by_dis['toc total']/teach_data_by_dis['Total']

teach_data_by_dis.head()


,STATE_DISTRICT_ID,White,Total,toc total,%_toc
0,10100010000,8033,9555,1522,0.159288
1,10100860829,74,123,49,0.398374
2,10100860830,97,132,35,0.265152
3,10100860867,98,201,103,0.512438
4,10100860892,121,154,33,0.214286


In [155]:



soc_df['teachers_of_color'] = np.nan
soc_df['total_teachers'] = np.nan
for i, l in enumerate(soc_df['id_list']):
    # test = teach_data_by_dis.groupby('STATE_DISTRICT_ID').apply(
    # lambda g: g[g['STATE_DISTRICT_ID'].isin(l)]).reset_index(drop=True)



    test = teach_data_by_dis.groupby('STATE_DISTRICT_ID').agg(
    filtered_sum_toc=('toc total', lambda x: x[x.index.isin(teach_data_by_dis[teach_data_by_dis['STATE_DISTRICT_ID'].isin(l)].index)].sum()),
    filtered_sum_total=('Total', lambda x: x[x.index.isin(teach_data_by_dis[teach_data_by_dis['STATE_DISTRICT_ID'].isin(l)].index)].sum()))
    test_cl = test[test['filtered_sum_toc']!=0]
    print(len(test_cl))
    toc = test_cl['filtered_sum_toc'].sum()
    total = test_cl['filtered_sum_total'].sum()

    soc_df.loc[i,['teachers_of_color']] = toc
    soc_df.loc[i,['total_teachers']] = total

soc_df['%_teachers_of_color'] = soc_df['teachers_of_color']/soc_df['total_teachers']

soc_df

2
2
2
3
2
32
2
3
6
3


,Region,soc_sum,total_sum,id_list,%_students_of_color,teachers_of_color,total_teachers,%_teachers_of_color
0,Capital District,15846,29602,"[10100010000, 10100010000, 10100010000, 101000...",0.535302,1593.0,15656.0,0.101750
1,Central New York,31170,42904,"[110200010000, 110200010000, 110200010000, 110...",0.726506,6001.0,21026.0,0.285409
2,Hudson Valley,13872,20754,"[131500010000, 131500010000, 131500010000, 131...",0.668401,1412.0,10765.0,0.131166
3,Long Island,9320,14698,"[280506060000, 280506060000, 280506060000, 280...",0.634100,1085.0,6897.0,0.157315
4,Mohawk Valley,14488,21050,"[412300010000, 412300010000, 412300010000, 412...",0.688266,1711.0,9382.0,0.182370
5,New York City,1474288,1764598,"[310100010000, 310100010000, 310100010000, 310...",0.835481,373095.0,838297.0,0.445063
6,North Country,682,5996,"[91200010000, 91200010000, 91200010000, 912000...",0.113742,774.0,3097.0,0.249919
7,The Finger Lakes,46838,58130,"[261600010000, 261600010000, 261600010000, 261...",0.805746,11737.0,34248.0,0.342706
8,The Southern Tier,14596,44424,"[30200010000, 30200010000, 30200010000, 302000...",0.328561,2555.0,23471.0,0.108858
9,Western New York,62298,84082,"[61700010000, 61700010000, 61700010000, 617000...",0.740920,6393.0,48390.0,0.132114


In [156]:


equity_df = soc_df.loc[:,['Region', '%_students_of_color', '%_teachers_of_color']]
equity_df.to_excel(r'C:\Users\clutz\OneDrive - THE HUNT INSTITUTE\Desktop\equity_gap_ny.xlsx', index=False)

In [ ]:



{"soc_sum":"sum of student counts that are not white (SUBGROUP CODE = 8) in the region",
 "total_sum":"sum of all students in the region",
 "percent":"sum of student counts that are not white (SUBGROUP CODE = 8)"
 }




In [ ]:



soc = pd.NamedAgg(column='Value', aggfunc=lambda x: x[x > 12].sum()),

item_day_count_v3 = df_w_regions['Region'].agg([
    ('total sum_calc', 'count'), 
    ('Pos_Return', lambda val: (val > 0).sum()), 
    ('Neg_Return', lambda val: (val < 0).sum()), 
    ('Mean', 'mean'), 
    ('Median', 'median'), 
    ('25%Percntile', percentile(25)), 
    ('75%Percntile', percentile(75)), 
    ('Max', 'max')
]).reset_index()


pd.group_by(['Region']).agg()


In [ ]:
#checks and testing
check = list(set(df_w_regions['Districts'].to_list()))
print(len(check))
print(len(names))


names_ref = list(set(names['Districts'].to_list()))

for n in names_ref:
    if n in check:
        continue
    else:
        print(f'didnt find {n}')

In [ ]:
soc_df = df[df['SUBGROUP CODE'] != 8]

soc_df.head(4)

# for i,(j,k) in enumerate(zip(df['SUBGROUP CODE'], df['SUBGROUP NAME'])):
#     # print(type(j))
#     if j == 8:
#         continue
        # print(f'white one here: {k}')

    

# soc_sum = 